In [ ]:
# %load /home/ds591/sandbox/geovista/examples/example_from_unstructured__lfric_orog_warp.py
import iris
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD

In [ ]:
import geovista as gv

In [ ]:
from aeolus.coord import isel
from aeolus.model import um

In [ ]:
import mypaths

In [ ]:
maindir = mypaths.results_lfric / "thai"

In [ ]:
sim_case = "ben1"
with_restart = False
rundir = f"run_lfric_atm_thai_{sim_case}{'with_restart' if with_restart else '_'}C24_dt-1800p0_intel_64-bit_fast-debug"

In [ ]:
fname = str(maindir / sim_case / "20" / rundir / "lfric_diag.nc")

In [ ]:
with PARSE_UGRID_ON_LOAD.context():
    cl = iris.load(fname)

In [ ]:
sfc_temp = isel(cl.extract_cube("surface_tile_temperature"), um.t, -1)[9, :]
u = isel(cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "u1_in_w3")), um.t, -1)
v = isel(cl.extract_cube(iris.Constraint(cube_func=lambda cube: cube.var_name == "u2_in_w3")), um.t, -1)

In [ ]:
wspd = u.copy(data=(u.data**2 + v.data**2)**0.5)
wspd.rename("wind_speed")

In [ ]:
mesh_sfc = gv.Transform.from_unstructured(
    *[i.points for i in sfc_temp.mesh.node_coords],
    sfc_temp.mesh.face_node_connectivity.indices_by_src(),
    data=sfc_temp.data,
    start_index=sfc_temp.mesh.face_node_connectivity.start_index,
    name=sfc_temp.name(),
)

mesh_uv_2d = gv.Transform.from_unstructured(
    *[i.points for i in u.mesh.node_coords],
    u.mesh.face_node_connectivity.indices_by_src(),
    start_index=wspd.mesh.face_node_connectivity.start_index,
    name="vectors",
)

# mesh.compute_normals(cell_normals=False, point_normals=True, inplace=True)
# mesh.warp_by_scalar(scalars="Normals", inplace=True, factor=0.1)

In [ ]:
plotter = gv.GeoPlotter()
plotter.add_mesh(mesh_sfc, show_scalar_bar=False, cmap="viridis", show_edges=False, edge_color="grey")
plotter.add_axes()
plotter.add_text(
    "Unstructured Cube-Sphere Node Data (N, 4)",
    position="upper_left",
    font_size=10,
    shadow=True,
)
plotter.show(jupyter_backend="panel")